In [ ]:

from abc import ABC, abstractmethod
import math
import time
from datetime import datetime
from scipy.stats import norm

date_string = "2025-09-29"
format_string = "%Y-%m-%d"
datetime_object = datetime.strptime(date_string, format_string)

class AbstractAsset(ABC):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price
    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'    
    
#class Stock:... 

class Stock(AbstractAsset):
    #def __init__(self, name : str, price : float):
    #    self.name = name
    #    self.price = price
    #
    __registry: dict[str, Stock] = dict()

    @classmethod
    def get_registry(cls, name: str) -> Stock:
        if (stock := cls.__registry.get(name)) in None:
            raise KeyError(f"Stock {name} doesn't exist")
        return stock

    def __new__(cls, *args, **kwargs):
        instance =  super().__new__(cls)
        name = kwargs.get('name')
        cls.__registry[name] = instance
        return instance
    #def __init__(self, name: str, price: float):
    #    self.name = name
    #    self.price = price
    #def __repr__(self) -> str:
    #   return f'Stock: {self.name}, \nPrice: {self.price}'

class Derivative(AbstractAsset):
    FORMAT_CODE = "%Y-%m-%d"
    def __init__(self, *args, expiration_date: str, underlying_name: str, **kwargs):
        super().__init__(*args, **kwargs)
        self.expiration_date = expiration_date
        self.__underlying =  Stock.get_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        return self.__underlying

class Option(Derivative):
    def __init__(self, *args, strike_price:float, risk_free_rate:float, volatility:float,  **kwargs):
        super().__init__(*args, **kwargs)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.__payoff = None
    
    @abstractmethod
    def exercise(self):...

    



class EuropeanCallOption(Option):
    @property
    def payoff(self) -> float:  #BSM call option
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T = T / (31536000)
        d1 = (math.log(self.underlying.price / self.strike_price) +\
            (self.risk_free_rate + self.volatility**2 / 2) * T) /\
            (self.volatility * math.sqrt(T))
        d2 = d1 - (self.volatility * math.sqrt(T))
        c = self.underlying.price * norm.cdf(d1) - self.strike_price * \
            math.exp(-self.risk_free_rate) * norm.cdf(d2)
        return c
    def exercise(self):
        return None #max(self.underlying.price - self.strike_price, 0)    #если дата экспирации не равна сегодняшней, то  
     
            


class AmericanCallOption:...




In [31]:
stock = Stock(name = 'VTBR', price = 70)

eur_call_option = EuropeanCallOption( 
    name = 'Default Name',
    strike_price = 70,
    risk_free_rate = 0.18,
    volatility = 0.3,
    expiration_date = '2025-12-31',
    price = 7,
    underlying_name = 'VTBR'
)

TypeError: argument of type 'NoneType' is not iterable

In [2]:
#1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ: (с урока)
#- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
#- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
#- Добавьте валидацию даты истечения в базовый класс `Option`
#- Создайте класс `Stock` (акция) как наследник `AbstractAsset`
from abc import ABC, abstractmethod
import math
import time
from datetime import datetime
from scipy.stats import norm

date_string = "2025-09-29"
format_string = "%Y-%m-%d"
datetime_object = datetime.strptime(date_string, format_string)

class AbstractAsset(ABC):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price

    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'    

class Stock(AbstractAsset):
    __registry: dict[str, 'Stock'] = dict()

    @classmethod
    def get_registry(cls, name: str) -> 'Stock':
        if (stock := cls.__registry.get(name)) is None:  # Исправлено: in -> is
            raise KeyError(f"Stock {name} doesn't exist")
        return stock

    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        name = kwargs.get('name')
        cls.__registry[name] = instance
        return instance
    
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price
    
    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'

class Derivative(AbstractAsset):
    FORMAT_CODE = "%Y-%m-%d"
    def __init__(self, *args, expiration_date: str, underlying_name: str, **kwargs):
        super().__init__(*args, **kwargs)
        self.expiration_date = expiration_date
        self.__underlying = Stock.get_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        return self.__underlying

class Option(Derivative):
    def __init__(self, *args, strike_price:float, risk_free_rate:float, volatility:float, **kwargs):
        super().__init__(*args, **kwargs)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.__payoff = None
    
    @abstractmethod
    def exercise(self):...

class EuropeanCallOption(Option):
    @property
    def payoff(self) -> float:  # BSM call option
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 31536000
        d1 = (math.log(self.underlying.price / self.strike_price) +\
              (self.risk_free_rate + self.volatility**2 / 2) * T) / \
             (self.volatility * math.sqrt(T))
        d2 = d1 - (self.volatility * math.sqrt(T))
        c = self.underlying.price * norm.cdf(d1) - self.strike_price * \
            math.exp(-self.risk_free_rate * T) * norm.cdf(d2)  # Исправлено: добавлен T
        return c
    def exercise(self):  # Исправлено: exerсise -> exercise (убрана кириллическая 'с')
        return None

vtbr = Stock(name = 'VTBR', price = 70)

class AmericanCallOption:...

stock = Stock(name='VTBR', price=70)

eur_call_option = EuropeanCallOption( 
    name='Default Name',
    strike_price=70,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=7,
    underlying_name='VTBR'
)
eur_call_option.payoff

5.702596625256909

In [ ]:
#Задание 1 БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ:** (доделанный)
#- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
#- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
#- Добавьте валидацию даты истечения в базовый класс `Option`
#- Создайте класс `Stock` (акция) как наследник `AbstractAsset`

from abc import ABC, abstractmethod
import math
import time
from datetime import datetime
from scipy.stats import norm

date_string = "2025-09-29"
format_string = "%Y-%m-%d"
datetime_object = datetime.strptime(date_string, format_string)

class AbstractAsset(ABC):
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price

    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'    

class Stock(AbstractAsset):
    __registry: dict[str, 'Stock'] = dict()

    @classmethod
    def get_registry(cls, name: str) -> 'Stock':
        if (stock := cls.__registry.get(name)) is None:  # Исправлено: in -> is
            raise KeyError(f"Stock {name} doesn't exist")
        return stock

    def __new__(cls, *args, **kwargs):
        instance = super().__new__(cls)
        name = kwargs.get('name')
        cls.__registry[name] = instance
        return instance
    
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = price
    
    def __repr__(self) -> str:
        return f'Stock: {self.name}, \nPrice: {self.price}'

class Derivative(AbstractAsset):
    FORMAT_CODE = "%Y-%m-%d"
    def __init__(self, *args, expiration_date: str, underlying_name: str, **kwargs):
        super().__init__(*args, **kwargs)
        self.expiration_date = expiration_date
        self.__underlying = Stock.get_registry(underlying_name)
    
    @property
    def underlying(self) -> Stock:
        return self.__underlying

class Option(Derivative):
    def __init__(self, *args, strike_price:float, risk_free_rate:float, volatility:float, **kwargs):
        super().__init__(*args, **kwargs)
        self.strike_price = strike_price
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.__payoff = None
    
    @abstractmethod
    def exercise(self):...

class EuropeanCallOption(Option):
    @property
    def payoff(self) -> float:  # BSM call option
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 31536000
        d1 = (math.log(self.underlying.price / self.strike_price) +\
              (self.risk_free_rate + self.volatility**2 / 2) * T) / \
             (self.volatility * math.sqrt(T))
        d2 = d1 - (self.volatility * math.sqrt(T))
        c = self.underlying.price * norm.cdf(d1) - self.strike_price * \
            math.exp(-self.risk_free_rate * T) * norm.cdf(d2)
        return c
    
    def exercise(self):  
        return None

class AmericanCallOption(Option):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__exercised = False
        self.__exercise_price = None
    
    @property
    def payoff(self) -> float:
        """Для американского колл-опциона используем биномиальную модель"""
        return self._binomial_tree_price()
    
    def _binomial_tree_price(self, steps: int = 100) -> float:
        """Реализация биномиальной модели для оценки американского опциона"""
        T = (datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()) / 31536000
        dt = T / steps
        u = math.exp(self.volatility * math.sqrt(dt))
        d = 1 / u
        p = (math.exp(self.risk_free_rate * dt) - d) / (u - d)
        
        # Создаем дерево цен базового актива
        prices = [[0.0] * (i + 1) for i in range(steps + 1)]
        prices[0][0] = self.underlying.price
        
        for i in range(1, steps + 1):
            for j in range(i + 1):
                prices[i][j] = self.underlying.price * (u ** (i - j)) * (d ** j)
        
        # Вычисляем выплаты на конечных узлах
        payoffs = [[0.0] * (i + 1) for i in range(steps + 1)]
        for j in range(steps + 1):
            payoffs[steps][j] = max(0, prices[steps][j] - self.strike_price)
        
        # Обратное индуктивное вычисление
        for i in range(steps - 1, -1, -1):
            for j in range(i + 1):
                continuation_value = math.exp(-self.risk_free_rate * dt) * (
                    p * payoffs[i + 1][j] + (1 - p) * payoffs[i + 1][j + 1]
                )
                exercise_value = max(0, prices[i][j] - self.strike_price)
                payoffs[i][j] = max(continuation_value, exercise_value)
        
        return payoffs[0][0]
    
    def exercise(self, current_price: float = None) -> float:
        """
        Исполнение американского опциона
        Возвращает прибыль от исполнения
        """
        if self.__exercised:
            raise ValueError("Option has already been exercised")
        
        if current_price is None:
            current_price = self.underlying.price
        
        payoff = max(0, current_price - self.strike_price)
        
        if payoff > 0:
            self.__exercised = True
            self.__exercise_price = current_price
            print(f"Option exercised at price {current_price}. Payoff: {payoff}")
        else:
            print("Exercise would result in zero payoff. Not exercising.")
        
        return payoff
    
    @property
    def is_exercised(self) -> bool:
        """Проверка, был ли опцион исполнен"""
        return self.__exercised
    
    @property
    def exercise_price(self) -> float:
        """Цена, по которой был исполнен опцион"""
        return self.__exercise_price

# Создаем акцию
vtbr = Stock(name='VTBR', price=70)

# Создаем европейский опцион
eur_call_option = EuropeanCallOption( 
    name='European VTBR Call',
    strike_price=70,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=7,
    underlying_name='VTBR'
)

# Создаем американский опцион
am_call_option = AmericanCallOption(
    name='American VTBR Call',
    strike_price=70,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=8,
    underlying_name='VTBR'
)

# Демонстрация работы
print("European Call Option Payoff:", eur_call_option.payoff)
print("American Call Option Payoff:", am_call_option.payoff)

# Исполнение американского опциона
print("\nExercising American Option:")
payoff1 = am_call_option.exercise(current_price=75)
print(f"Payoff when exercising at 75: {payoff1}")

# Попытка исполнения еще раз
try:
    payoff2 = am_call_option.exercise(current_price=80)
except ValueError as e:
    print(f"Error: {e}")

print(f"Is option exercised: {am_call_option.is_exercised}")
print(f"Exercise price: {am_call_option.exercise_price}")

European Call Option Payoff: 5.742216916620357
American Call Option Payoff: 5.731680016603852

Exercising American Option:
Option exercised at price 75. Payoff: 5
Payoff when exercising at 75: 5
Error: Option has already been exercised
Is option exercised: True
Exercise price: 75


In [41]:
#**2. РАСШИРЕННЫЕ ОПЦИОНЫ:**
#- Реализуйте класс `BinaryOption` (бинарный опцион)
#- Создайте класс `BarrierOption` (барьерный опцион)
from enum import Enum

class OptionType(Enum):
    CALL = "call"
    PUT = "put"
class BarrierType(Enum):
    UP_AND_IN = "up_and_in"
    UP_AND_OUT = "up_and_out"
    DOWN_AND_IN = "down_and_in"
    DOWN_AND_OUT = "down_and_out"

class BinaryOption(Option):
    """
    Бинарный опцион (опцион "все или ничего")
    Выплачивает фиксированную сумму если условие выполняется
    """
    def __init__(self, *args, option_type: OptionType, payout_amount: float = 100.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.option_type = option_type
        self.payout_amount = payout_amount
    
    @property
    def payoff(self) -> float:
        """
        Расчет стоимости бинарного опциона по формуле Блэка-Шоулза
        """
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 31536000
        
        d2 = (math.log(self.underlying.price / self.strike_price) + 
              (self.risk_free_rate - self.volatility**2 / 2) * T) / \
             (self.volatility * math.sqrt(T))
        
        if self.option_type == OptionType.CALL:
            # Бинарный колл: выплата если цена выше страйка
            return self.payout_amount * math.exp(-self.risk_free_rate * T) * norm.cdf(d2)
        else:
            # Бинарный пут: выплата если цена ниже страйка
            return self.payout_amount * math.exp(-self.risk_free_rate * T) * norm.cdf(-d2)
    
    def exercise(self, current_price: float = None) -> float:
        """
        Исполнение бинарного опциона
        Возвращает фиксированную выплату если условие выполняется
        """
        if current_price is None:
            current_price = self.underlying.price
        
        if self.option_type == OptionType.CALL:
            # Для колла: выплата если текущая цена выше страйка
            payoff = self.payout_amount if current_price > self.strike_price else 0.0
        else:
            # Для пута: выплата если текущая цена ниже страйка
            payoff = self.payout_amount if current_price < self.strike_price else 0.0
        
        print(f"Binary Option Exercise - Type: {self.option_type.value}, "
              f"Current Price: {current_price}, Strike: {self.strike_price}, "
              f"Payoff: {payoff}")
        
        return payoff

class BarrierOption(Option):
    """
    Барьерный опцион - активируется или деактивируется при достижении барьера
    """
    def __init__(self, *args, option_type: OptionType, barrier_type: BarrierType, 
                 barrier_level: float, **kwargs):
        super().__init__(*args, **kwargs)
        self.option_type = option_type
        self.barrier_type = barrier_type
        self.barrier_level = barrier_level
        self.__activated = self._check_initial_activation()
    
    def _check_initial_activation(self) -> bool:
        """
        Проверка начальной активации барьерного опциона
        """
        current_price = self.underlying.price
        
        if "in" in self.barrier_type.value:
            # Опцион активируется при достижении барьера
            if "up" in self.barrier_type.value:
                return current_price >= self.barrier_level
            else:  # down
                return current_price <= self.barrier_level
        else:
            # Опцион деактивируется при достижении барьера
            return True
    
    @property
    def is_activated(self) -> bool:
        """Проверка активирован ли опцион"""
        return self.__activated
    
    def update_barrier_status(self, current_price: float = None) -> bool:
        """
        Обновление статуса барьера на основе текущей цены
        """
        if current_price is None:
            current_price = self.underlying.price
        
        old_status = self.__activated
        
        if "in" in self.barrier_type.value:
            # Для knock-in опционов
            if "up" in self.barrier_type.value and current_price >= self.barrier_level:
                self.__activated = True
            elif "down" in self.barrier_type.value and current_price <= self.barrier_level:
                self.__activated = True
        else:
            # Для knock-out опционов
            if "up" in self.barrier_type.value and current_price >= self.barrier_level:
                self.__activated = False
            elif "down" in self.barrier_type.value and current_price <= self.barrier_level:
                self.__activated = False
        
        if old_status != self.__activated:
            print(f"Barrier status changed: {old_status} -> {self.__activated}")
        
        return self.__activated
    
    @property
    def payoff(self) -> float:
        """
        Расчет стоимости барьерного опциона
        Используем аналитическую формулу для европейского стиля
        """
        if not self.__activated:
            return 0.0
        
        # Для упрощения используем цену обычного европейского опциона
        # В реальности здесь должна быть специализированная формула для барьерных опционов
        T: float = datetime.strptime(self.expiration_date, self.FORMAT_CODE).timestamp() - time.time()
        T /= 31536000
        
        d1 = (math.log(self.underlying.price / self.strike_price) + 
              (self.risk_free_rate + self.volatility**2 / 2) * T) / \
             (self.volatility * math.sqrt(T))
        d2 = d1 - self.volatility * math.sqrt(T)
        
        if self.option_type == OptionType.CALL:
            base_value = self.underlying.price * norm.cdf(d1) - \
                        self.strike_price * math.exp(-self.risk_free_rate * T) * norm.cdf(d2)
        else:
            base_value = self.strike_price * math.exp(-self.risk_free_rate * T) * norm.cdf(-d2) - \
                        self.underlying.price * norm.cdf(-d1)
        
        # Применяем барьерный дисконт
        barrier_discount = 0.7 if "out" in self.barrier_type.value else 1.3
        
        return base_value * barrier_discount
    
    def exercise(self, current_price: float = None) -> float:
        """
        Исполнение барьерного опциона
        """
        if current_price is None:
            current_price = self.underlying.price
        
        # Обновляем статус барьера
        self.update_barrier_status(current_price)
        
        if not self.__activated:
            print(f"Barrier option is not activated. Type: {self.barrier_type.value}")
            return 0.0
        
        # Расчет выплаты как у обычного опциона
        if self.option_type == OptionType.CALL:
            payoff = max(0, current_price - self.strike_price)
        else:
            payoff = max(0, self.strike_price - current_price)
        
        print(f"Barrier Option Exercise - Type: {self.option_type.value}, "
              f"Barrier Type: {self.barrier_type.value}, Current Price: {current_price}, "
              f"Strike: {self.strike_price}, Barrier: {self.barrier_level}, "
              f"Payoff: {payoff}")
        
        return payoff

# Демонстрация работы расширенных опционов

# Создаем акцию
vtbr = Stock(name='VTBR', price=70)

# Бинарный колл опцион
binary_call = BinaryOption(
    name='VTBR Binary Call',
    strike_price=75,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=45,
    underlying_name='VTBR',
    option_type=OptionType.CALL,
    payout_amount=100.0
)

print(f"Binary Call Payoff Value: {binary_call.payoff:.2f}")
print(f"Binary Call Exercise at 80: {binary_call.exercise(80)}")
print(f"Binary Call Exercise at 70: {binary_call.exercise(70)}")

# Бинарный пут опцион
binary_put = BinaryOption(
    name='VTBR Binary Put',
    strike_price=65,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=40,
    underlying_name='VTBR',
    option_type=OptionType.PUT,
    payout_amount=100.0
)

print(f"\nBinary Put Payoff Value: {binary_put.payoff:.2f}")
print(f"Binary Put Exercise at 60: {binary_put.exercise(60)}")
print(f"Binary Put Exercise at 70: {binary_put.exercise(70)}")

print("\n=== БАРЬЕРНЫЕ ОПЦИОНЫ ===")
# Up-and-In барьерный колл
barrier_up_in = BarrierOption(
    name='VTBR Up-In Call',
    strike_price=70,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=6,
    underlying_name='VTBR',
    option_type=OptionType.CALL,
    barrier_type=BarrierType.UP_AND_IN,
    barrier_level=80.0
)

print(f"Up-In Barrier Initial Status: {barrier_up_in.is_activated}")
print(f"Up-In Barrier Payoff: {barrier_up_in.payoff:.2f}")

# Симулируем достижение барьера
print("\n--- Testing barrier activation ---")
barrier_up_in.update_barrier_status(85.0)  # Цена превышает барьер
print(f"Up-In Barrier Status after price 85: {barrier_up_in.is_activated}")
print(f"Up-In Barrier Payoff after activation: {barrier_up_in.payoff:.2f}")

# Up-and-Out барьерный колл
barrier_up_out = BarrierOption(
    name='VTBR Up-Out Call',
    strike_price=70,
    risk_free_rate=0.18,
    volatility=0.3,
    expiration_date='2025-12-31',
    price=8,
    underlying_name='VTBR',
    option_type=OptionType.CALL,
    barrier_type=BarrierType.UP_AND_OUT,
    barrier_level=80.0
)

print(f"\nUp-Out Barrier Initial Status: {barrier_up_out.is_activated}")
print(f"Up-Out Barrier Payoff: {barrier_up_out.payoff:.2f}")

# Симулируем достижение барьера (деактивация)
barrier_up_out.update_barrier_status(85.0)  # Цена превышает барьер
print(f"Up-Out Barrier Status after price 85: {barrier_up_out.is_activated}")
print(f"Up-Out Barrier Payoff after deactivation: {barrier_up_out.payoff:.2f}")

# Тестирование исполнения
print("\n--- Testing barrier option exercise ---")
print(f"Up-In Exercise at 75: {barrier_up_in.exercise(75)}")
print(f"Up-Out Exercise at 75: {barrier_up_out.exercise(75)}")



Binary Call Payoff Value: 38.70
Binary Option Exercise - Type: call, Current Price: 80, Strike: 75, Payoff: 100.0
Binary Call Exercise at 80: 100.0
Binary Option Exercise - Type: call, Current Price: 70, Strike: 75, Payoff: 0.0
Binary Call Exercise at 70: 0.0

Binary Put Payoff Value: 22.51
Binary Option Exercise - Type: put, Current Price: 60, Strike: 65, Payoff: 100.0
Binary Put Exercise at 60: 100.0
Binary Option Exercise - Type: put, Current Price: 70, Strike: 65, Payoff: 0.0
Binary Put Exercise at 70: 0.0

=== БАРЬЕРНЫЕ ОПЦИОНЫ ===
Up-In Barrier Initial Status: False
Up-In Barrier Payoff: 0.00

--- Testing barrier activation ---
Barrier status changed: False -> True
Up-In Barrier Status after price 85: True
Up-In Barrier Payoff after activation: 7.46

Up-Out Barrier Initial Status: True
Up-Out Barrier Payoff: 4.02
Barrier status changed: True -> False
Up-Out Barrier Status after price 85: False
Up-Out Barrier Payoff after deactivation: 0.00

--- Testing barrier option exercise ---

In [ ]:
#Задание 3  ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ:**
#- Реализуйте метод `__repr__` для класса `Portfolio`
#- Добавьте метод `__getitem__` для доступа к активам по индексу
#- Реализуйте метод `__contains__` для проверки наличия актива в портфеле
#- Добавьте метод `remove_asset` для удаления актива из портфеля

class Portfolio:
    def __init__(self):
        self.assets = []
    
    def __repr__(self):
        return f"Portfolio({self.assets})"
    
    def __getitem__(self, index):
        return self.assets[index]
    
    def __contains__(self, asset):
        return asset in self.assets
    
    def remove_asset(self, asset):
        if asset in self.assets:
            self.assets.remove(asset)
    
    # демонстрация работы методов
    def add_asset(self, asset):
        self.assets.append(asset)

# Пример использования
if __name__ == "__main__":
    portfolio = Portfolio()
    portfolio.add_asset("AAPL")
    portfolio.add_asset("GOOGL")
    portfolio.add_asset("TSLA")
    
    print(portfolio)           
    print(portfolio[1])       
    print("AAPL" in portfolio) 
    portfolio.remove_asset("GOOGL")
    print(portfolio)          

Portfolio(['AAPL', 'GOOGL', 'TSLA'])
GOOGL
True
Portfolio(['AAPL', 'TSLA'])


In [ ]:
#Задание 4 - pass

In [ ]:
#Задание 5 ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ:**
#- Добавьте `__eq__`, `__lt__`, `__le__` для сравнения портфелей
#- Реализуйте `__mul__` для умножения портфеля на число (масштабирование)
#- Добавьте `__str__` для красивого вывода портфеля
#- Реализуйте `__bool__` для проверки, является ли портфель пустым

class Portfolio:
    def __init__(self, assets=None):
        self.assets = assets if assets is not None else []
    
    def __getitem__(self, key):
        """Доступ к активам по индексу или срезу"""
        if isinstance(key, slice):
            return Portfolio(self.assets[key])
        return self.assets[key]
    
    def __contains__(self, asset):
        """Проверка наличия актива в портфеле"""
        return any(
            a.name == asset.name and a.price == asset.price 
            for a in self.assets
        )
    
    def __eq__(self, other):
        """Сравнение портфелей по общей стоимости"""
        if not isinstance(other, Portfolio):
            return False
        return self.total_value() == other.total_value()
    
    def __lt__(self, other):
        """Сравнение 'меньше' по общей стоимости"""
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self.total_value() < other.total_value()
    
    def __le__(self, other):
        """Сравнение 'меньше или равно' по общей стоимости"""
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self.total_value() <= other.total_value()
    
    def __mul__(self, factor):
        """Умножение портфеля на число (масштабирование цен)"""
        if not isinstance(factor, (int, float)):
            return NotImplemented
        
        scaled_assets = []
        for asset in self.assets:
            # Создаем новый актив с масштабированной ценой
            if isinstance(asset, Stock):
                scaled_assets.append(Stock(asset.price * factor, asset.name))
            elif isinstance(asset, Cash):
                scaled_assets.append(Cash(asset.amount * factor, asset.currency))
        
        return Portfolio(scaled_assets)
    
    def __rmul__(self, factor):
        """Умножение числа на портфель (для коммутативности)"""
        return self.__mul__(factor)
    
    def __str__(self):
        """Красивый вывод портфеля для пользователя"""
        if not self.assets:
            return "Портфель пуст"
        
        result = ["Портфель активов:"]
        result.append("-" * 30)
        
        for i, asset in enumerate(self.assets, 1):
            result.append(f"{i}. {asset}")
        
        result.append("-" * 30)
        result.append(f"Общая стоимость: ${self.total_value():.2f}")
        
        return "\n".join(result)
    
    def __bool__(self):
        """Проверка на пустоту портфеля"""
        return len(self.assets) > 0
    
    def total_value(self):
        """Общая стоимость портфеля"""
        return sum(asset.price if hasattr(asset, 'price') else asset.amount 
                  for asset in self.assets)
    
    def add_asset(self, asset):
        """Добавление актива в портфель"""
        self.assets.append(asset)


# Вспомогательные классы для демонстрации
class Stock:
    def __init__(self, price, name):
        self.price = price
        self.name = name
    
    def __str__(self):
        return f"Акция '{self.name}' - Цена: ${self.price:.2f}"
    
    def __repr__(self):
        return f"Stock({self.price}, '{self.name}')"
    
    def __eq__(self, other):
        if not isinstance(other, Stock):
            return False
        return self.name == other.name and self.price == other.price


class Cash:
    def __init__(self, amount, currency="USD"):
        self.amount = amount
        self.currency = currency
    
    def __str__(self):
        return f"Наличные: {self.amount:.2f} {self.currency}"
    
    def __repr__(self):
        return f"Cash({self.amount}, '{self.currency}')"
    
    def __eq__(self, other):
        if not isinstance(other, Cash):
            return False
        return self.amount == other.amount and self.currency == other.currency


# Пример использования:
if __name__ == "__main__":
    # Создаем портфель
    pf = Portfolio([Stock(150.50, 'AAPL'), Stock(2750.25, 'GOOGL'), Cash(5000)])
    
    print("=== Тестирование методов ===")
    
    # __getitem__
    print("Первый актив:", pf[0])
    print("Срез активов:", pf[1:3])
    
    # __contains__
    aapl_stock = Stock(150.50, 'AAPL')
    print(f"Акция {aapl_stock} в портфеле:", aapl_stock in pf)
    
    # __eq__, __lt__, __le__
    pf2 = Portfolio([Stock(150.50, 'AAPL'), Stock(2750.25, 'GOOGL'), Cash(5000)])
    pf3 = Portfolio([Stock(100, 'TSLA')])
    
    print("pf == pf2:", pf == pf2)
    print("pf == pf3:", pf == pf3)
    print("pf < pf3:", pf < pf3)
    print("pf <= pf2:", pf <= pf2)
    
    # __mul__
    doubled_pf = pf * 2
    print("\nУдвоенный портфель:")
    print(doubled_pf)
    
    # __rmul__
    tripled_pf = 3 * pf
    print("\nУтроенный портфель:")
    print(tripled_pf)
    
    # __str__
    print("\nКрасивый вывод портфеля:")
    print(pf)
    
    # __bool__
    empty_pf = Portfolio()
    print("pf не пустой:", bool(pf))
    print("empty_pf не пустой:", bool(empty_pf))

=== Тестирование методов ===
Первый актив: Акция 'AAPL' - Цена: $150.50
Срез активов: Портфель активов:
------------------------------
1. Акция 'GOOGL' - Цена: $2750.25
2. Наличные: 5000.00 USD
------------------------------
Общая стоимость: $7750.25
Акция Акция 'AAPL' - Цена: $150.50 в портфеле: True
pf == pf2: True
pf == pf3: False
pf < pf3: False
pf <= pf2: True

Удвоенный портфель:
Портфель активов:
------------------------------
1. Акция 'AAPL' - Цена: $301.00
2. Акция 'GOOGL' - Цена: $5500.50
3. Наличные: 10000.00 USD
------------------------------
Общая стоимость: $15801.50

Утроенный портфель:
Портфель активов:
------------------------------
1. Акция 'AAPL' - Цена: $451.50
2. Акция 'GOOGL' - Цена: $8250.75
3. Наличные: 15000.00 USD
------------------------------
Общая стоимость: $23702.25

Красивый вывод портфеля:
Портфель активов:
------------------------------
1. Акция 'AAPL' - Цена: $150.50
2. Акция 'GOOGL' - Цена: $2750.25
3. Наличные: 5000.00 USD
--------------------------